In [36]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle
df = pd.read_csv('listings_summary.csv')

In [37]:
# Data contains 22552 rows and 96 columns and no duplicate values
df.shape
df.duplicated().sum()

0

In [38]:
#Convert the boolean columns to numeric values
features_having_boolean = ["require_guest_phone_verification", "host_is_superhost", "host_has_profile_pic",
                           "host_identity_verified", "is_location_exact", "requires_license", "instant_bookable"]

def boolean_to_numeric(data):
    if data == "t" or data == "T":
        return 1.0
    elif data == "f" or data == "F":
        return 0.0
    else:
        return None
    
for i in features_having_boolean:
    df[i] = df[i].map(boolean_to_numeric)
    

In [39]:
# Converts amenities to amenities count
pd.options.mode.chained_assignment = None
length = len(df["amenities"])
for i in range(0, length):
    value = (df["amenities"][i]).split(",")
    df["amenities"][i] = len(value)

In [40]:
columns_to_keep = ['id','space', 'host_has_profile_pic', 'neighbourhood_group_cleansed', 
                   'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',  
                   'bedrooms', 'bed_type', 'amenities', 'square_feet', 'price', 'cleaning_fee', 
                   'security_deposit', 'extra_people', 'guests_included', 'minimum_nights',  
                   'instant_bookable', 'is_business_travel_ready', 'cancellation_policy', "is_location_exact"]

df = df[columns_to_keep]

In [41]:
# Clean price value
price_columns = ["price", "extra_people","cleaning_fee", "security_deposit"]
for col in price_columns:
    for i in range(0, len(df)):
        if not(isinstance(df[col][i],float)):
            df[col][i] = (df[col][i]).replace("$", "")
            df[col][i] = (df[col][i]).replace(",", "")

In [42]:
# Replacing the null price values to 0
# There are pleanty of null values that means the charges are more likely 0
# Cleaning fee - 7146 null values
# Security deposit - 9361 null values

obj_cols = ['cleaning_fee', 'security_deposit']
for col in obj_cols:
    df[col] = df[col].astype(float)
    df[col].fillna(0, inplace=True)  

In [43]:
null_cols = df.columns[df.isna().any()].tolist()   
df.isna().sum()  

id                                  0
space                            8532
host_has_profile_pic               26
neighbourhood_group_cleansed        0
latitude                            0
longitude                           0
property_type                       0
room_type                           0
accommodates                        0
bathrooms                          32
bedrooms                           18
bed_type                            0
amenities                           0
square_feet                     22106
price                               0
cleaning_fee                        0
security_deposit                    0
extra_people                        0
guests_included                     0
minimum_nights                      0
instant_bookable                    0
is_business_travel_ready            0
cancellation_policy                 0
is_location_exact                   0
dtype: int64

In [44]:
# Drop columns with too many Null values
df.drop(columns=['square_feet', 'space'], inplace=True)

In [45]:
# Handle the null values using mean value
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()    
numeric_df = df[numeric_cols]
null_cols = numeric_df.columns[numeric_df.isna().any()].tolist()
print("Null values repalced by mean:", null_cols)
for col in null_cols:
      df[col] = df[col].mean()
   

Null values repalced by mean: ['host_has_profile_pic', 'bathrooms', 'bedrooms']


In [46]:
# Handle longitude and latitude
def distance_from_berlin(lat, lon):
    berlin_centre = (52.5027778, 13.404166666666667)
    record = (lat, lon)
    return great_circle(berlin_centre, record).km

#add distanse dataset
df['distance'] = df.apply(lambda x: distance_from_berlin(x.latitude, x.longitude), axis=1)

del df['latitude']
del df['longitude']